<a href="https://colab.research.google.com/github/SBiscardi/cd-202202-santiago-biscardi/blob/master/Taller9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import statsmodels.api as sm
from scipy.special import logit
from sklearn.model_selection import train_test_split
from math import e
import numpy as np


In [ ]:
dfOriginal = pd.read_csv("https://raw.githubusercontent.com/adiacla/bigdata/master/titanic.csv")
dfOriginal['Survived'] = dfOriginal['Survived'].fillna(0)
dfOriginal['Sex'] = dfOriginal['Sex'].replace('male', 0)
dfOriginal['Sex'] = dfOriginal['Sex'].replace('female', 1)

In [ ]:
# Cabin tiene muchos valores nulos
df = dfOriginal.copy(deep=True)
df = df.drop('Cabin', axis= 1)

In [ ]:
# Borrar los nulos
df = df.dropna()

In [ ]:
dfTrain, dfTest = train_test_split(df, test_size=0.2, random_state=25)

In [ ]:
# Hacer regresión de age, siblings y parents vs sobrevivencia
x = dfTrain.iloc[:,[5, 6, 7]]
y = dfTrain.Survived
logit_model = sm.Logit(y,x)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.663036
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  569
Model:                          Logit   Df Residuals:                      566
Method:                           MLE   Df Model:                            2
Date:                Sun, 06 Nov 2022   Pseudo R-squ.:                 0.01419
Time:                        03:17:57   Log-Likelihood:                -377.27
converged:                       True   LL-Null:                       -382.70
Covariance Type:            nonrobust   LLR p-value:                  0.004375
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Age           -0.0141      0.003     -4.950      0.000      -0.020      -0.009
SibSp         -0.2017      0.

In [ ]:
ageCoef = result.params.values[0]
siblingsCoef = result.params.values[1]
parentsCoef = result.params.values[2]

In [ ]:
dfTest['PredictedValue'] = np.nan
dfTest.index = range(dfTest.shape[0])

for i in range(dfTest.shape[0]):
  pred = 1/(1+e**((ageCoef*dfTest.loc[i, 'Age']) + (siblingsCoef*dfTest.loc[i, 'SibSp']) + (parentsCoef*dfTest.loc[i, 'Parch'])))
  value = 0
  if pred > 0.5:
    value = 1
  elif pred is not None:
    value = 0
  
  dfTest['PredictedValue'][i] = value


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
dfTest['PredictedValue'].value_counts()

1.0    125
0.0     18
Name: PredictedValue, dtype: int64

In [ ]:
dfTest['Survived'].value_counts()

0    82
1    61
Name: Survived, dtype: int64

In [ ]:
dfTest[['Age', 'SibSp', 'Parch', 'Survived', 'PredictedValue']]

,Age,SibSp,Parch,Survived,PredictedValue
0,3.00,1,1,1,0.0
1,0.92,1,2,1,0.0
2,19.00,0,0,0,1.0
3,10.00,3,2,0,1.0
4,31.00,0,0,1,1.0
...,...,...,...,...,...
138,22.00,0,0,0,1.0
139,24.00,1,0,1,1.0
140,4.00,3,2,0,1.0
141,50.00,1,0,0,1.0


In [ ]:
counter = 0
for i in range(dfTest.shape[0]):
  if dfTest.loc[i, 'Survived'] == dfTest.loc[i, 'PredictedValue']:
    counter+=1

percentage = (counter/dfTest.shape[0])*100

In [ ]:
print(f'El porcentaje de prediccion es de: {percentage}')

El porcentaje de prediccion es de: 37.06293706293706


El porcentaje de prediccion para las pruebas es muy bajo, el algoritmo no es totalmente confiable, en mis pruebas sobrevive mucho mas gente que la que realmente sobrevivio

In [ ]:
dfTrain, dfTest = train_test_split(df, test_size=0.2, random_state=25)

In [ ]:
# Hacer regresión de age, siblings y parents vs sobrevivencia
x = dfTrain.iloc[:,[4, 5, 6, 7]]
y = dfTrain.Survived
logit_model = sm.Logit(y,x)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.538392
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  569
Model:                          Logit   Df Residuals:                      565
Method:                           MLE   Df Model:                            3
Date:                Sun, 06 Nov 2022   Pseudo R-squ.:                  0.1995
Time:                        03:19:01   Log-Likelihood:                -306.35
converged:                       True   LL-Null:                       -382.70
Covariance Type:            nonrobust   LLR p-value:                 6.864e-33
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Sex            2.3661      0.224     10.582      0.000       1.928       2.804
Age           -0.0320      0.

In [ ]:
sexCoef = result.params.values[0]
ageCoef = result.params.values[1]
siblingsCoef = result.params.values[2]
parentsCoef = result.params.values[3]

In [ ]:
dfTest['PredictedValue'] = np.nan
dfTest.index = range(dfTest.shape[0])

for i in range(dfTest.shape[0]):
  pred = 1/(1+e**((sexCoef*dfTest.loc[i, 'Sex'] + ageCoef*dfTest.loc[i, 'Age']) + (siblingsCoef*dfTest.loc[i, 'SibSp']) + (parentsCoef*dfTest.loc[i, 'Parch'])))
  value = 0
  if pred > 0.5:
    value = 1
  elif pred is not None:
    value = 0
  
  dfTest['PredictedValue'][i] = value


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
dfTest['PredictedValue'].value_counts()

1.0    87
0.0    56
Name: PredictedValue, dtype: int64

In [ ]:
dfTest['Survived'].value_counts()

0    82
1    61
Name: Survived, dtype: int64

In [ ]:
counter = 0
for i in range(dfTest.shape[0]):
  if dfTest.loc[i, 'Survived'] == dfTest.loc[i, 'PredictedValue']:
    counter+=1

percentage = (counter/dfTest.shape[0])*100

In [ ]:
print(f'El porcentaje de prediccion es de: {percentage}')

El porcentaje de prediccion es de: 20.27972027972028


El porcentaje de prediccion para las pruebas es muy bajo, el algoritmo no es totalmente confiable, en mis pruebas los valores son similares, pero no corresponden a la persona que es